# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [18]:
import pandas as pd
import numpy as np
from sklearn import datasets
import pkg_resources
import azureml.core
from azureml.widgets import RunDetails
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.datastore import Datastore
from azureml.pipeline.steps import AutoMLStep
from azureml.core.model import Model, InferenceConfig
from azureml.core.resource_configuration import ResourceConfiguration
from azureml.core import Environment
from azureml.core.webservice import AciWebservice
import requests
import json

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'diabetes-automl'

experiment=Experiment(ws, experiment_name)

In [4]:
dataset = Dataset.get_by_name(ws, name='diabetes')
dataset.to_pandas_dataframe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
cluster_name = "mimi-compute"

try:
    aml_compute = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',
                                                           max_nodes=4)
    aml_compute = ComputeTarget.create(ws, cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Found existing cluster
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [7]:
# TODO: Put your automl settings here
automl_settings = { "experiment_timeout_minutes": 30,
                    "max_concurrent_iterations": 5,
                    "primary_metric" : 'accuracy'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=aml_compute,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Outcome",   
                             path = '.',
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [8]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
diabetes-automl,AutoML_d8b32f9a-7233-4396-a964-ee5c0498a578,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [11]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [13]:
auto_job, best_auto_model = remote_run.get_output()
print(auto_job)
print(best_auto_model)

Package:azureml-automl-runtime, training version:1.50.0, current version:1.49.0
Package:azureml-core, training version:1.50.0, current version:1.49.0
Package:azureml-dataprep, training version:4.10.6, current version:4.9.1
Package:azureml-dataprep-rslex, training version:2.17.5, current version:2.16.1
Package:azureml-dataset-runtime, training version:1.50.0, current version:1.49.0
Package:azureml-defaults, training version:1.50.0, current version:1.49.0
Package:azureml-interpret, training version:1.50.0, current version:1.49.0
Package:azureml-mlflow, training version:1.50.0, current version:1.49.0
Package:azureml-pipeline-core, training version:1.50.0.post1, current version:1.49.0
Package:azureml-responsibleai, training version:1.50.0, current version:1.49.0
Package:azureml-telemetry, training version:1.50.0, current version:1.49.0
Package:azureml-train-automl-client, training version:1.50.0, current version:1.49.0
Package:azureml-train-automl-runtime, training version:1.50.0, current 

Run(Experiment: diabetes-automl,
Id: AutoML_d8b32f9a-7233-4396-a964-ee5c0498a578_38,
Type: azureml.scriptrun,
Status: Completed)
Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                 PreFittedSoftVotingClassifier(classification_labels=array([0, 1]), estimators=[('14', Pipeline(memory=None, steps=[('standardscalerwrapper', StandardScalerWrapper(copy=True, with_mean=False, with_std=False)), ('xgboostclassifier', XGBoostClassifier(booster='gbtree', colsample_bytree=1, eta=0.3, gamma=0, max_depth=10, max_leaves=511, n_estimators=10, n_jobs=1, objective='reg:logistic', problem_info=ProblemInfo(gpu_training_param_dict={'processing_unit_type': 

In [14]:
import joblib

# Save the best model
joblib.dump(value=best_auto_model, filename='auto_model.joblib')

['auto_model.joblib']

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [16]:
model = auto_job.register_model(model_name='best-automl-model',
                                      model_path='outputs/model.pkl',
                                      description='Best model from AutoML for diabetes')

TODO: In the cell below, send a request to the web service you deployed to test it.

In [17]:
auto_job.download_files(output_directory='automl_output')

env = Environment.from_conda_specification('automl-env', './automl_output/outputs/conda_env_v_1_0_0.yml')

inference = InferenceConfig(environment=env, entry_script='./automl_output/outputs/scoring_file_v_1_0_0.py')

deployment = AciWebservice.deploy_configuration(cpu_cores=1,
                                                       memory_gb=1,
                                                       enable_app_insights=True)

deployment_name = 'auto-diabetes'
service = Model.deploy(workspace=ws,
                       name=deployment_name,
                       models=[model],
                       inference_config=inference,
                       deployment_config=deployment,
                       overwrite=True)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-05-10 18:21:30+00:00 Creating Container Registry if not exists..
2023-05-10 18:31:31+00:00 Registering the environment..
2023-05-10 18:31:32+00:00 Building image..
2023-05-10 18:51:48+00:00 Generating deployment configuration.
2023-05-10 18:51:49+00:00 Submitting deployment to compute..
2023-05-10 18:51:54+00:00 Checking the status of deployment auto-diabetes..
2023-05-10 18:53:57+00:00 Checking the status of inference endpoint auto-diabetes.
Succeeded
ACI service creation operation finished, operation "Succeeded"


TODO: In the cell below, print the logs of the web service and delete the service

In [19]:
rest_endpoint = service.scoring_uri
headers = {'Content-type': 'application/json'}
response = requests.post(rest_endpoint,
                         headers=headers,
                         json={
  "data": [
    {
      "Pregnancies": 4,
      "Glucose": 121,
      "BloodPressure": 70,
      "SkinThickness": 21,
      "Insulin": 80,
      "BMI": 32.0,
      "DiabetesPedigreeFunction": 0.47,
      "Age": 33
    }
  ]
}
                        )
response.content


b'"{\\"result\\": [0]}"'

In [22]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

data =  {
  "data": [
    {
      "Pregnancies": 4,
      "Glucose": 121,
      "BloodPressure": 70,
      "SkinThickness": 21,
      "Insulin": 80,
      "BMI": 32.0,
      "DiabetesPedigreeFunction": 0.47,
      "Age": 33
    }
  ],
  "method": "predict"
}

body = str.encode(json.dumps(data))

url = 'http://01b8aaff-c40d-4b46-adb9-c1d426f1367b.westus2.azurecontainer.io/score'


headers = {'Content-Type':'application/json'}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print(result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

    # Print the headers - they include the requert ID and the timestamp, which are useful for debugging the failure
    print(error.info())
    print(error.read().decode("utf8", 'ignore'))

b'"{\\"result\\": [0]}"'


In [21]:
logs = service.get_logs()
for line in logs.split('\n'):
    print(line)

/bin/bash: /azureml-envs/azureml_399903ab218c6bc60c3ac2efb6ffbdcd/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_399903ab218c6bc60c3ac2efb6ffbdcd/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_399903ab218c6bc60c3ac2efb6ffbdcd/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_399903ab218c6bc60c3ac2efb6ffbdcd/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-05-10T18:53:37,780608600+00:00 - iot-server/run 
2023-05-10T18:53:37,781116200+00:00 - rsyslog/run 
2023-05-10T18:53:37,788882800+00:00 - gunicorn/run 
2023-05-10T18:53:37,790244000+00:00 | gunicorn/run | 
2023-05-10T18:53:37,797163200+00:00 | gunicorn/run | ###############################################
2023-05-10T18:53:37,799422700+00:00 | gunicorn/run | AzureML Container Runtime Information
2023-05-10T18:53:37,806076600

In [ ]:
# Delete the Webservice and Compute Cluster
service.delete()
compute_target.delete()

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.
